#Problem

# Puzzle-and-Dragon-Max-Combos
An exercise trying to find the max. combos on 6*5 grid

Given a 6 * 5 grid G that every index contains a random element from 6 different type (say A,B,C,D,E,F), there will be N waves to destroy the elements, until no more elements can be destroyed. 

The elements can only be destroyed when there appears with 3 or more consecutive elements from same type horizontally or vertically. Matches with more than 3 elements that in same type will be considered as 1 combo. For instance, a 2 x 3 clearance / a horizontal column of 3 and a vertical row of 3 that are connected by an element in the middle, will be considered as 1 combo. The system will scan through the grid from (0,0) to (4,5) on row-first basis, i.e. (0,0) -> (0,1) -> (0,2) -> …  -> (0,5) -> (1,0) -> (1,1) -> … -> (4,5) for 1 wave. After each wave, the remaining elements will be dropped vertically and start the next wave, no elements will be filled after each wave. 

With given G, player can rearrange the elements with any possible arrangement for initial round, but the total number of each element types must remain the same after the arrangement. 

Example flow, 

1. 0 combos

AAACDF

BDCCEF

BDEEEE

BBBCDF

CDAAAF

2. 4 combos

AAACDF

BDCCEF

BDEEEE

BBBCDF

CDAAAF


XXXXXX

XXXXXF

XDXCDF

XDXCEF

CDCCDF

3. 4 + 3, 7 combos total

XXXXXX

XXXXXF

XDXCDF

XDXCEF

CDCCDF


XXXXXX

XXXXXX

XXXXDX

XXXXEX

CXCXDX

The goal is to find the best initial arrangement in order to get highest possible number of combos with either approach from below:

	1. Design an algorithm to find the arrangement .
	2. Use Machine Learning to train the model to find the arrangement.

*Reference: http://pad.wikia.com/wiki/Game_Mechanics

In [1]:
import numpy as np
import itertools


In [2]:
# create a empty grid with all values as -1
def create_empty_grid():
    return np.full((5, 6), -1)

# get score and find duplicated matches
def find_score(grid):
    matches = []
    
    # find all the 3-combo matches first
    for x in range(0, 5):
        for y in range(0, 6):
            e1 = grid[x][y]
            # vertical (orientation = 0)
            if x < 3:
                e2 = grid[x + 1][y]
                e3 = grid[x + 2][y]
                if e1 == e2 and e1 == e3:
                    matches.append((e1, x, y, 0))
            # horizontal (orientation = 1)
            if y < 4:
                e2 = grid[x][y + 1]
                e3 = grid[x][y + 2]
                if e1 == e2 and e1 == e3:
                    matches.append((e1, x, y, 1))
                    
    match_grid = create_empty_grid()
    score = 0
    
    # try to remove duplicated matches in the list
    for (e_type, x, y, orientation) in matches:
        for i in range(0, 3):
            is_in_match = False
            # vertical (orientation = 0)
            if orientation == 0:
                e1 = match_grid[x][y]
                e2 = match_grid[x + 1][y]
                e3 = match_grid[x + 2][y]
                
                # check if they are used already, if used -> connected with other matches, 
                # we should remove it from the matches
                if e1 == -1:
                    match_grid[x][y] = e_type
                else:
                    is_in_match = True
                if e2 == -1:
                    match_grid[x + 1][y] = e_type
                else:
                    is_in_match = True
                if e3 == -1:
                    match_grid[x + 2][y] = e_type
                else:
                    is_in_match = True
                    
                # if that is not connected with other matches, score + 1
                if not is_in_match:
                    score = score + 1
                    
            # horizontal (orientation = 1)
            else:
                e1 = match_grid[x][y]
                e2 = match_grid[x][y + 1]
                e3 = match_grid[x][y + 2]
                if e1 == -1:
                    match_grid[x][y] = e_type
                else:
                    is_in_match = True
                if e2 == -1:
                    match_grid[x][y + 1] = e_type
                else:
                    is_in_match = True
                if e3 == -1:
                    match_grid[x][y + 2] = e_type
                else:
                    is_in_match = True

                if not is_in_match:
                    score = score + 1
    return score

In [3]:
# This function is used to fetch number of each elements in the grid
def get_element_numbers(grid):
    numbers = [0, 0, 0, 0, 0, 0]
    for x in range(0, 5):
        for y in range(0, 6):
            element = grid[x][y]
            numbers[element] = numbers[element] + 1
    return numbers

# Max. score in theory 
def find_score_in_simple_way(grid):
    numbers = [int(n / 3) for n in get_element_numbers(grid)]
    return sum(numbers)

#Analyzing the problem

With above requirements, we can easily found that the max. combos for each grid will only be 10. Because the will only be 6 * 5 = 30 of elements, and in most ideal case we can destroy the elements with 3 consecutive elements, which is equal to 30 / 3 = 10. 

From here we can further simplify the problem. Given that we can relocate the elements with any arrangement we want, the initial arrangement can’t provide useful information for us now. All we need to know is the number of different elements. Take above pattern as an example, we can simplify it to a 1-D array A: {6, 5, 5, 5, 5, 4}, which equals to the total numbers of the array {A, B, C, D, E, F}. Since there are no possible ways to achieve higher combos with insufficient elements, we can simply find the sum of A % 3, which is 2 + 1 + 1 + 1 + 1 + 1 = 7 in this case. 

Note that above algorithm is only suitable when it happens to be a relatively even distribution. When a single element has a total number = 15, it reaches its maximum combos, which is 5. See below as an example,

AXAXAA

AXAXAA

AXAXAA

XXXXXX

AAAXXX

When we try to add A on the into the grid, it will not increase the no, of combos but with a possibility to connect the matches and hence reduce the number of combos. 


Base on this, we build a simple method to test the score of the grid, named "find_score_in_simple_way".

However, when number of A >= 21, there will only be 9 or less free spaces for us to insert the A, that will not be possible to not connect any existing matches. Therefore, starting from A = 21, the combos will decrease with the rise of number of A.

Number of A = 20, example grid:

AXAXAA

AXAXAA

AXAXAA

XAXAXX

AAAAAA

Therefore, we used find_score method to identify connected elements and remove duplicated matches fomr the grid. 


After setting the score method, we now want to find the possible inital arrangement for the board. To obtain this goal, we have serval methods. 

1. Brute-force all the possible permutation of 5 * 6 gird for 6 elements. In this case, it will probably take forever to load since the possible number of permutation would be 6^30 = 2.210739197+e23.

2. Method in 1 contains many duplicated and unnecessary arrangements, which the method is very inefficient and unnecessary. As we know the number of each elements are fixed and the elements are identical with in same group. we can find the permutation to be : $\prod_0^{30} n$ for n being the number of distinct elements. For simple case like A=24, B=6, C=0, D=0, E=0, F=0, the permutation would be 2^24 * 1^6 = 16,777,216. We can use a Depth-First search to implement it. However, it will still suffer from average case where elements are evenly distributed.

3. To further improve the algorithm, we want to group the elements as 3-combos sets becuase it is eventually what we want. Excessive permutations from method 2 would contains many arrangements that they are clearly impossible to gain max. combos. Worst case would have 10! = 3628800 permutations but most of them are duplicates, we can use DFS to remove the duplicates we want



In [11]:
# Use itertools to get permutations but contains duplicates
def find_permutations__itertools(numbers):
    gen = itertools.permutations(numbers, len(numbers))
    return list(gen)


# Use Depth-First Search to find unique permutations  
def find_unqiue_permutations__DFS(numbers):
    res = []
    dfs(res, [], sorted(numbers))
    return res


def dfs(res, path, numbers):
    if not numbers:
        res.append(path)
    for i in range(len(numbers)):
        if i > 0 and numbers[i] == numbers[i-1]:
            continue
        dfs(res, path + [numbers[i]], numbers[:i] + numbers[i+1:])

In [5]:
# Group the elements in 3, first group the ones that can form combos, i.e., in same element type.
# Afterwards group randomly.
def group_elements(grid):
    # Flatten the 2d-array to 1d 
    lst = np.sort(grid.flatten())
    element_groups = []
    # find combos first
    while len(lst) > 2:
        e1 = lst[0]
        e2 = lst[1]
        e3 = lst[2]
        if e1 == e2 and e1 == e3:
            element_groups.append([e1, e2, e3])
            lst = np.delete(lst, [0, 1, 2])
        else:
            element_groups.append([e1])
            lst = np.delete(lst, 0)
    if len(lst) > 0:
        for i in range(0, len(lst)):
            element_groups.append([lst[i]])
    
    temp_list = []
    i = 0
    # group the single lists randomly and remove them from the original list
    while i < len(element_groups):
        if len(element_groups[i]) == 1:
            temp_list.append(element_groups[i][0])
            del element_groups[i]
            i = i - 1
        if len(temp_list) == 3:
            element_groups.append(temp_list)
            temp_list = []
        i = i + 1
    return element_groups

In [6]:
# Helper function to transfer the grouped elements back to grid
def grouped_elements_to_grid(grouped_elements):
    temp_list = []
    for i in range(0, len(grouped_elements), 2):
        temp_list.append(np.hstack([grouped_elements[i], grouped_elements[i + 1]]))
    return np.array(temp_list)

In [7]:
# some data testing
grid = np.random.randint(6, size=(5, 6))

grid2 = np.array([[0, 0, 1, 1, 1, 1],
                  [0, 0, 1, 0, 0, 0],
                  [0, 0, 1, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0]
                  ])

print(grid)
print(grid2)

# !!!!!! DO NOT RUN THIS LINE, IT TAKES FOREVER     
# numbers = np.sort(grid.flatten())     
# result = find_permutations__itertools(numbers)    
# print("Done with Method 1!")   
# print(len(result))    
# print(result[1])  
    
# result = find_unqiue_permutations(numbers)    
# print("Done with Method 2!")   
# print(len(result))    
# print(result[1])  
    
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!   

In [90]:
grouped_elements = group_elements(grid)
gen_result = find_permutations__itertools(grouped_elements)
print("Done with Method 3!")
print(len(gen_result))
print(gen_result[10])

gen_result = find_unqiue_permutations__DFS(grouped_elements)
print("Done with Method 3 - DFS!")
print(len(gen_result))
print(gen_result[10])

# test grouped_elements_to_grid 
print(grouped_elements_to_grid(gen_result[511]))
print(find_score_in_simple_way(grouped_elements_to_grid(gen_result[511])))


([0, 0, 0], [0, 0, 0], [0, 0, 0], [1, 1, 1], [1, 1, 1], [2, 2, 2], [5, 5, 5], [3, 4, 5], [4, 4, 4], [0, 2, 3])


#Conclusion

By obversing and thinking, we now sized the problem to a less complex one. Hence, we significantly reduced the computation time to a reasonable time now. The DFS permutation would only produce 7!~8! on average cases, which is around 5040 ~ 40320. We have a working function below to integrated everything we have so far. 

It first take the grid and transform it to 3-elements grouped list, which has the size 10. From these 10 groups, we go through a DFS permutation to find all the unique permutations. And then we will get its highest possible score in theory using find_score_in_simple_way. Finally we loop through the permutations and find the best grid arrangements. 

#Future work

Due to limited time, this solution is only a very simple one compare to real P&D system. There are few things we may want to improve in the future:

1. Integrated the elimination step beolow into DFS so that we can stop running DFS once we found the solution.

2. Introduce weighting systems to calculate score, for instances, five/four-element combos will have a higher socre than 3-element one, or element 6 has higher weight than element 2etc.

4. Add move systems that would allow users to swap elements.

3. Try to fit machine learning algorithms such as supervised / reinforcement learning into the project and let the machine decide the best arragenment

In [9]:
def solution(grid):
    grouped_elements = group_elements(grid)
    perms = find_unqiue_permutations__DFS(grouped_elements)
    max_possible_score = find_score_in_simple_way(grid)
    best_perm = []
    best_score = 0
    
    # Stop once we find the grid with highest possible socre
    for p in perms:
        arrangement = grouped_elements_to_grid(p)
        score = find_score(arrangement)
        if score == max_possible_score:
            return arrangement
        if score > best_score:
            best_perm = arrangement
            best_score = score
            
    return best_perm

In [14]:
final_result = solution(grid)
print("Best Arrangement: \n", final_result)
print("Best Score: ", find_score(final_result))

Best Arrangement: 
 [[0 0 0 0 0 1]
 [1 1 1 1 2 3]
 [2 2 2 3 3 3]
 [2 2 2 3 4 5]
 [4 4 4 5 5 5]]
Best Score:  7
